# Solution: The Tool Error

This notebook provides the complete solution to the debug drill.

---

In [ ]:
import json
from typing import Dict, Any

In [ ]:
# Mock order database
ORDER_DATABASE = {
    "ORD-001": {"status": "delivered", "date": "2024-01-15"},
    "ORD-002": {"status": "shipped", "date": "2024-01-17"},
}

def get_order_status(order_id: str) -> Dict[str, Any]:
    if order_id not in ORDER_DATABASE:
        return {"error": f"Order {order_id} not found"}
    return {"order_id": order_id, **ORDER_DATABASE[order_id]}

In [ ]:
# ===== BROKEN CODE =====

def broken_agent_respond(query: str, order_id: str) -> str:
    tool_result = get_order_status(order_id)
    
    # BUG: Hallucinates when tool fails
    if "error" in tool_result:
        return f"Your order {order_id} shipped yesterday and should arrive soon."
    else:
        return f"Your order {order_id} is {tool_result['status']}."

print("=== Broken Agent ===")
print(f"ORD-001: {broken_agent_respond('Status?', 'ORD-001')}")
print(f"ORD-003: {broken_agent_respond('Status?', 'ORD-003')} ❌ HALLUCINATED!")

In [ ]:
# ===== SOLUTION: Fixed agent with error handling =====

def fixed_agent_respond(query: str, order_id: str) -> str:
    """Agent that properly handles tool errors."""
    tool_result = get_order_status(order_id)
    
    # Check for errors FIRST
    if "error" in tool_result:
        return f"I couldn't find order {order_id} in our system. Please double-check the order number."
    
    # Only respond with data when tool succeeds
    return f"Your order {order_id} is currently {tool_result['status']} as of {tool_result['date']}."

print("=== Fixed Agent ===")
print(f"ORD-001: {fixed_agent_respond('Status?', 'ORD-001')}")
print(f"ORD-003: {fixed_agent_respond('Status?', 'ORD-003')} ✓ Honest!")

In [ ]:
# ===== SOLUTION: Safe tool executor =====

class SafeToolExecutor:
    """Tool executor that validates results before using them."""
    
    def __init__(self):
        self.call_log = []
    
    def execute(self, tool_name: str, **kwargs) -> Dict[str, Any]:
        tools = {"get_order_status": get_order_status}
        
        if tool_name not in tools:
            return {"error": f"Unknown tool: {tool_name}", "success": False}
        
        try:
            result = tools[tool_name](**kwargs)
        except Exception as e:
            result = {"error": str(e)}
        
        result["success"] = "error" not in result
        
        self.call_log.append({
            "tool": tool_name,
            "args": kwargs,
            "success": result["success"]
        })
        
        return result
    
    def format_for_llm(self, result: Dict) -> str:
        if not result["success"]:
            return f"TOOL_ERROR: {result['error']}\nDo not make up information."
        return f"TOOL_SUCCESS: {json.dumps({k: v for k, v in result.items() if k != 'success'})}"

executor = SafeToolExecutor()

print("=== Safe Executor ===")
r1 = executor.execute("get_order_status", order_id="ORD-001")
print(f"ORD-001: {executor.format_for_llm(r1)}")

r2 = executor.execute("get_order_status", order_id="ORD-003")
print(f"ORD-003: {executor.format_for_llm(r2)}")

In [ ]:
# Verify solution
response = fixed_agent_respond("Status?", "ORD-003")
assert "shipped" not in response.lower()
assert "couldn't find" in response.lower()
print("✓ Agent no longer hallucinates on tool errors!")

## Solution Summary

**Problem:** Agent says "shipped yesterday" for non-existent order

**Solution:**
1. **Check for errors first** before generating response
2. **Admit failures honestly** instead of making things up
3. **Format errors clearly** for LLM with explicit instructions

**Key principle:** Never let tool errors pass silently—the LLM will fill the gap with hallucination